<a href="https://colab.research.google.com/github/denzilsaldanha/monument/blob/master/RESNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import glob
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam




base_model=ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3) )

x=base_model.output

# x = Flatten()(x)
# x = Dense(1024, activation="relu")(x)
# x = Dropout(0.5)(x)
# x = Dense(1024, activation="relu")(x)
# predictions = Dense(150, activation="softmax")(x)
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
# x=Dense(1024,activation='relu')(x) #dense layer 2
# x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(150,activation='softmax')(x) #final layer with softmax activation


# In[3]:


model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture


# In[4]:

layers_to_be_trained = 0 
for layer in model.layers[:158]:
    layer.trainable=False
for layer in model.layers[158:]:
    layer.trainable=True
    layers_to_be_trained +=1

print(layers_to_be_trained)
# In[5]:


# train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,validation_split=0.2)
# train_generator=train_datagen.flow_from_directory('./train/', # this is where you specify the path to the main data folder
#                                                  target_size=(224,224),
#                                                  color_mode='rgb',
#                                                  batch_size=32,
#                                                  class_mode='categorical',
#                                                  shuffle=True)


training_set = train_datagen.flow_from_directory('drive/My Drive/GOOGLE_MONUMENT/train_sample_images',
                                                 target_size = (224, 224),
                                                 batch_size = 128,
                                                 class_mode = 'categorical',
                                                subset ='training')
validation_set = train_datagen.flow_from_directory(
    'drive/My Drive/GOOGLE_MONUMENT/train_sample_images', # same directory as training data
    target_size=(224,224),
    batch_size=128,
    class_mode='categorical',
    subset='validation')

# In[33]:
epochs_run = 0

# if glob.glob('drive/My Drive/GOOGLE_MONUMENT/weights-improvement-resnet-[0-9][0-9]*'):
#   weight_to_be_loaded = max(glob.glob('drive/My Drive/GOOGLE_MONUMENT/weights-improvement-resnet-[0-9][0-9]*'))
#   model.load_weights(weight_to_be_loaded)
#   epochs_run = int(weight_to_be_loaded[58:60])

model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy


csv_file = 'drive/My Drive/GOOGLE_MONUMENT/log_for_res_net1_starting_from_epoch_'+str(epochs_run)+'.csv'
checkpoint="drive/My Drive/GOOGLE_MONUMENT/weights-improvement-resnet1-{epoch:02d}-{acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(checkpoint, monitor='val_acc', verbose=1, mode='max')
csv_logger = CSVLogger(csv_file, append=True, separator=';')
callbacks_list = [csv_logger,checkpoint]

#model.summary()
step_size_train=training_set.n//training_set.batch_size
val_steps = validation_set.n//validation_set.batch_size
#print(val_steps)
model.fit_generator(generator=training_set,
                   steps_per_epoch=step_size_train,
                   epochs=5,
                    validation_data = validation_set,validation_steps=val_steps,
                   callbacks =callbacks_list,initial_epoch = epochs_run)


#print("%s: %.2f%%" % (classifier.metrics_names[1], scores[1]*100))
model.save('drive/My Drive/GOOGLE_MONUMENT/model_resnet1.hdf5')
model.save_weights("drive/My Drive/GOOGLE_MONUMENT/model_h5_resnet1.h5")
print("Saved model to disk") 
# serialize model to JSON
model_json = model.to_json()
with open("drive/My Drive/GOOGLE_MONUMENT/model_resnet1.json", "w") as json_file:
    json_file.write(model_json)


